In [3]:
import sys
sys.path.insert(0,'../../')
import time
from facebook_graph_api import *
import pandas as pd




FORMAT = '%(asctime)-15s %(clientip)s %(user)-8s %(message)s'
logging.basicConfig(level=logging.ERROR)
LOGGER.info("business discovery...")

In [4]:
import os
print(os.getcwd())

/home/anh_lbt/atvn/sw_follou/recommender_api/test


In [62]:
from recommend_utils import recommend_utils, constants
from database import models
from datetime import datetime
import time
from database import db_cassandra
from cassandra.query import dict_factory
from FacebookCrawler import facebook_graph_api
from cassandra.cqlengine import columns, connection

class watch_list():
    def __init__(self):
        # #get session
        
        self.fb_graph = facebook_graph_api.FacebookGraphApi('../FacebookCrawler/instagram_token.csv')
        (self.fb_graph.access_token,self.fb_graph.user_id) = constants.TOKENS[4]
        self.db = db_cassandra.db_cassandra(contact_points = ['172.33.47.9'], factory = dict_factory)
        connection.set_session(self.db.session)


    def get_wl_by_user_id(self, user_id):
        try:
            query_string="select * from aify_recommend.watch_list where user_id = '%s'"%user_id
            re = self.db.session.execute(query_string)
            list_dict_ = recommend_utils.map_to_dict_factory_1(re.current_rows)#data get from db, list have only one item
        #     list_dict_ = re.current_rows
            return list_dict_[0]
        except Exception as ex:
            return None

    def wl_add(self,user_id, ig_id):
        try:
            wl_item = self.fb_graph.get_business_info(username = ig_id)
            wl_item['new_post'] = 0
            wl = self.get_wl_by_user_id(user_id)
            if wl is None:
                wl = models.WatchList(user_id = user_id, watch_user_list = {ig_id:models.type_business_info(**wl_item)}, viewed_time = datetime.utcnow(),updated_time = datetime.utcnow())
                wl.save()
            else:
                if wl['watch_user_list'] is None:
                    models.WatchList.objects(user_id = user_id).update(watch_user_list= {ig_id:models.type_business_info(**wl_item)})
                else:   
                    if ig_id in wl['watch_user_list']:
                        print("user %s already in watch list"%ig_id)
                        return False
                    else:
                        wl['watch_user_list'][ig_id] = models.type_business_info(**wl_item)     
                        models.WatchList.objects(user_id = user_id).update(watch_user_list= wl['watch_user_list'])
            return True
            
        except Exception as ex:
            print(ex)
            return False

    def wl_remove(self,user_id, ig_id):
        try:
            wl = self.get_wl_by_user_id(user_id)
            if wl is None:
                return False
            elif wl['watch_user_list'] is not None and ig_id in wl['watch_user_list']:
                #print('remove %s '%ig_id)
                del wl['watch_user_list'][ig_id]
                models.WatchList.objects(user_id = user_id).update(watch_user_list= None)
                models.WatchList.objects(user_id = user_id).update(watch_user_list= wl['watch_user_list'])
                return True
            else:
                print('no see %s in database'%ig_id)
                return False
        except Exception as ex:
            print(ex)
            return False    
        
        

    def wl_update_new_post(self,user_id):
    #     list_infos = []
        try:
            wl = self.get_wl_by_user_id(user_id)
            if wl is None:
                return False
            elif wl['watch_user_list'] is not None:
                for k, username in enumerate(wl['watch_user_list']):
            #     for k, v in list_dict_[0]['watch_user_list'].items(): # iter dictionary
                    print(username)
                    request_data = self.fb_graph.get_business_info(username = username)
        #             list_infos.append(request_data)
                    #
                    num_new_post = request_data['media_count'] - int(wl['watch_user_list'][username].media_count)
                    request_data['new_post'] = num_new_post
                    if num_new_post != 0:
                        #update db
                        wl['watch_user_list'][username] = models.type_business_info(**request_data) #update item in watch list user
                        print('you have %s new post'%num_new_post)
                        return True
                    else:
                        print('no changes')
                        return False
            else:
                print('no have any user in watch list, please add first...')
                return False
                    
            models.WatchList.objects(user_id = user_id).update(watch_user_list= wl['watch_user_list'])      

        except Exception as ex:
            print(ex)
            
    # watch_list.objects(user_id = 'anhlbt').update(watch_user_list= {'natje_9999': a2, 'emmaparkerandco': a2})   

    # update_string = "update watch_list set watch_user_list= {'natje_9999': 'a2', 'emmaparkerandco': 'a1'} where user_id = 'anhlbt' if exists"
    # _session.execute(update_string)

    # wl = watch_list(user_id = 'anhlbt3',watch_user_list= a['watch_user_list'], viewed_time = datetime.utcnow(),updated_time = datetime.utcnow())
    # wl.save()


if __name__ == '__main__':
    wl_inst = watch_list()
    a = wl_inst.wl_add(user_id='anhlbt', ig_id='natje_9999')
    print(a)
    

/home/anh_lbt/atvn/sw_follou/recommender_api/test
user natje_9999 already in watch list
False


In [65]:
wl_inst.wl_remove(user_id = 'anhlbt', ig_id='natje_9999')

True

In [120]:
a = wl_inst.get_wl_by_user_id(user_id='anhlbt')

In [119]:
wl_inst.wl_add(user_id='anhlbt', ig_id='emmaparkerandco')

True

In [63]:
wl_inst.wl_update_new_post(user_id='anhlbt')

natje_9999
no changes


False

In [121]:
def convert_info_objects_to_json(info_object):
    tmp_dict = info_object['watch_user_list']
    for k, v in tmp_dict.items():
        print(k)
        tmp_dict[k] = dict(tmp_dict[k])
    return info_object

In [123]:
b = convert_info_objects_to_json(a)

emmaparkerandco
natje_9999


In [124]:
b

{'user_id': 'anhlbt',
 'updated_time': datetime.datetime(2018, 8, 24, 7, 6, 46, 555000),
 'viewed_time': datetime.datetime(2018, 8, 24, 7, 6, 46, 555000),
 'watch_user_list': {'emmaparkerandco': {'id': '17841401690883742',
   'website': 'https://www.emmaparkerdiamonds.com/',
   'media_count': 330,
   'followers_count': 647,
   'name': 'Emma Parker & Co.',
   'username': 'emmaparkerandco',
   'profile_picture_url': 'https://scontent.xx.fbcdn.net/v/t51.2885-15/12393747_142605859440768_1709991462_a.jpg?_nc_cat=0&oh=8b7b47ecf00a2e0e865d2c0d06993e63&oe=5BF30F51',
   'new_post': 0},
  'natje_9999': {'id': '17841401660142798',
   'website': 'https://bit.ly/2v0yJDU',
   'media_count': 805,
   'followers_count': 1696,
   'name': 'Nathalie Madsen',
   'username': 'natje_9999',
   'profile_picture_url': 'https://scontent.xx.fbcdn.net/v/t51.2885-15/33933220_493197567763039_6170514771826180096_n.jpg?_nc_cat=0&oh=4d09397c07c3f0d0667f8688d02a4e65&oe=5BF17C73',
   'new_post': 0}}}